In [1]:
import pyspark as ps
spark = (ps.sql.SparkSession.builder 
        .master("local[4]") 
        .appName("yelp_academic") 
        .getOrCreate()
        )
sc = spark.sparkContext

In [2]:
from train_model import str_to_l, if_rest_udf, str_to_list_udf, construct_set_for_ALS

In [19]:
from train_model import hash_udf

In [3]:
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd

In [4]:
from pyspark.ml.recommendation import ALS
from pyspark.sql.types import *
from pyspark.sql.functions import udf

In [22]:
from pymongo import MongoClient

mc = MongoClient()
db  = mc['raw_restaurants']
bisn = db['restaurants']
users = db['users']
rv = db['reviews']
rv_s = db['reviews_scrap']

In [5]:
biz = spark.read.json('yelp_dataset/yelp_academic_dataset_business.json')

rev = spark.read.json('yelp_dataset/yelp_academic_dataset_review.json')

user = spark.read.json('yelp_dataset/yelp_academic_dataset_user.json')

<i> Next cell is not need for now </i>

In [24]:
biz.createOrReplaceTempView('business')
rev.createOrReplaceTempView('reviews')

In [27]:
biz.count(), rev.count()

(188593, 5996996)

In [6]:
rests = biz.filter(if_rest_udf(biz.categories))

In [7]:
rests.cache()

DataFrame[address: string, attributes: struct<AcceptsInsurance:string,AgesAllowed:string,Alcohol:string,Ambience:string,BYOB:string,BYOBCorkage:string,BestNights:string,BikeParking:string,BusinessAcceptsBitcoin:string,BusinessAcceptsCreditCards:string,BusinessParking:string,ByAppointmentOnly:string,Caters:string,CoatCheck:string,Corkage:string,DietaryRestrictions:string,DogsAllowed:string,DriveThru:string,GoodForDancing:string,GoodForKids:string,GoodForMeal:string,HairSpecializesIn:string,HappyHour:string,HasTV:string,Music:string,NoiseLevel:string,Open24Hours:string,OutdoorSeating:string,RestaurantsAttire:string,RestaurantsCounterService:string,RestaurantsDelivery:string,RestaurantsGoodForGroups:string,RestaurantsPriceRange2:string,RestaurantsReservations:string,RestaurantsTableService:string,RestaurantsTakeOut:string,Smoking:string,WheelchairAccessible:string,WiFi:string>, business_id: string, categories: string, city: string, hours: struct<Friday:string,Monday:string,Saturday:string

In [79]:
#rests = rests.withColumn('categories', str_to_list_udf(rests.categories))

In [93]:
##Wanna try two ways
rests.createOrReplaceTempView('rests')
rev.createOrReplaceTempView('reviews')

In [7]:
rest_id = rests.select('business_id')

In [9]:
rest_id.createOrReplaceTempView('rests_id')

In [98]:
%%time
select = '''
SELECT  reviews.* 
FROM rests_id 
LEFT JOIN reviews ON rests_id.business_id == reviews.business_id


'''
rest_rev = spark.sql(select)
rest_rev.show(5)

+--------------------+----+----------+-----+--------------------+-----+--------------------+------+--------------------+
|         business_id|cool|      date|funny|           review_id|stars|                text|useful|             user_id|
+--------------------+----+----------+-----+--------------------+-----+--------------------+------+--------------------+
|--9e1ONYQuAa-CB_R...|   0|2014-02-25|    0|8eEp26oqTXmuIjVUJ...|    3|It was a real up ...|     0|AdxhlxXSGjRsCQxG6...|
|--9e1ONYQuAa-CB_R...|   0|2017-02-14|    0|VETXTwMw6qxzOVDlX...|    5|went for dinner t...|     0|ymlnR8UeFvB4FZL56...|
|--9e1ONYQuAa-CB_R...|   0|2013-12-19|    0|NQt645IJwa5ADM26U...|    5|we are a repeat c...|     0|TMqFvYbWqs8BnjLsE...|
|--9e1ONYQuAa-CB_R...|   0|2013-12-02|    2|G9TSEZ8BfuSmcf5xy...|    3|Thanksgiving dinn...|     5|KmubWHd-t2d3dxLZ1...|
|--9e1ONYQuAa-CB_R...|   0|2015-01-01|    0|6prMgAjp2LZjuxRvm...|    4|We were seated pr...|     0|bMFqD4beGrt_LjNGG...|
+--------------------+----+-----

In [8]:
%%time
rest_rev2 = rest_id.join(rev, "business_id")
rest_rev2.show(5)

+--------------------+----+----------+-----+--------------------+-----+--------------------+------+--------------------+
|         business_id|cool|      date|funny|           review_id|stars|                text|useful|             user_id|
+--------------------+----+----------+-----+--------------------+-----+--------------------+------+--------------------+
|iCQpiavjjPzJ5_3gP...|   0|2011-02-25|    0|x7mDIiDB3jEiPGPHO...|    2|The pizza was oka...|     0|msQe1u7Z_XuqjGoqh...|
|pomGBqfbxcqPv14c3...|   0|2012-11-13|    0|dDl8zu1vWPdKGihJr...|    5|I love this place...|     0|msQe1u7Z_XuqjGoqh...|
|jtQARsP6P-LbkyjbO...|   1|2014-10-23|    1|LZp4UX5zK3e-c5ZGS...|    1|Terrible. Dry cor...|     3|msQe1u7Z_XuqjGoqh...|
|elqbBhBfElMNSrjFq...|   0|2011-02-25|    0|Er4NBWCmCD4nM8_p1...|    2|Back in 2005-2007...|     2|msQe1u7Z_XuqjGoqh...|
|Ums3gaP2qM3W1XcA5...|   0|2014-09-05|    0|jsDu6QEJHbwP2Blom...|    5|Delicious healthy...|     0|msQe1u7Z_XuqjGoqh...|
+--------------------+----+-----

### Join faster??? after lunch try in diff order

In [99]:
rest_rev2.count()

3654797

In [11]:
rest_rev2.cache()

DataFrame[business_id: string, cool: bigint, date: string, funny: bigint, review_id: string, stars: bigint, text: string, useful: bigint, user_id: string]

Baseline --- predict just mean rest rating

In [12]:
base_pred = rests.select('business_id', 'stars').withColumnRenamed('stars',
'mean_rating').join(rev.select('business_id', 'stars'), 'business_id')

In [13]:
baseline_rdd =base_pred.rdd


In [119]:
r1 = baseline_rdd.first()
r1[0], r1[1],r1[2]

('iCQpiavjjPzJ5_3gPD5Ebg', 4.0, 2)

In [118]:
baseline_rdd.cache()

MapPartitionsRDD[263] at javaToPython at NativeMethodAccessorImpl.java:0

In [120]:
sum_sq = baseline_rdd.map(lambda x : (x[2] - x[1])**2).sum()
num = baseline_rdd.count()


In [138]:
from math import sqrt
sum_sq/num, sqrt(sum_sq/num)

(1.5400588596302338, 1.240991079593336)

In [165]:
def look_how_dence_rating_matrix(rev, m):
    rating = rev.select('user_id', 'business_id', 'stars')
    user_id = rating.select('user_id').groupBy('user_id').count().selectExpr('user_id', 'count AS user_count')
    
    filt_rat = user_id.filter(user_id.user_count>m).join(rating, 'user_id')
    u  = user_id.filter(user_id.user_count>m).count()
    v = filt_rat.select('business_id').groupBy('business_id').count().count()
    r = filt_rat.count()
    k = r/(u+v)
    return u, v, r, k

In [166]:
look_how_dence_rating_matrix(rest_rev2,10)

(56425, 55341, 1611480, 14.418338313977417)

In [167]:
%%time
m_k = {}
for m in range(20):
    m_k[m] = look_how_dence_rating_matrix(rest_rev2,m)

CPU times: user 496 ms, sys: 184 ms, total: 680 ms
Wall time: 16min 26s


In [168]:
m_k

{0: (1031840, 57173, 3654797, 3.356063701718896),
 1: (464515, 57049, 3087472, 5.919641693061638),
 2: (292072, 56861, 2742586, 7.859921532213921),
 3: (207596, 56683, 2489158, 9.41867496093144),
 4: (157816, 56483, 2290038, 10.686181456749681),
 5: (125431, 56287, 2128113, 11.71107430194037),
 6: (103142, 56090, 1994379, 12.52498869573955),
 7: (86606, 55915, 1878627, 13.181404845601701),
 8: (74139, 55737, 1778891, 13.696841602759555),
 9: (64410, 55537, 1691330, 14.100644451299324),
 10: (56425, 55341, 1611480, 14.418338313977417),
 11: (50120, 55161, 1542125, 14.647704714050969),
 12: (44882, 55003, 1479269, 14.80972117935626),
 13: (40438, 54841, 1421497, 14.919310656073216),
 14: (36842, 54667, 1371153, 14.98380487165197),
 15: (33691, 54504, 1323888, 15.010918986337094),
 16: (30959, 54338, 1280176, 15.008452817801329),
 17: (28556, 54197, 1239325, 14.976194216523872),
 18: (26490, 54054, 1202137, 14.925220997218911),
 19: (24724, 53929, 1168583, 14.857449811196012)}

In [9]:
rest_rev2.columns

['business_id',
 'cool',
 'date',
 'funny',
 'review_id',
 'stars',
 'text',
 'useful',
 'user_id']

In [10]:
rating = rest_rev2.select('business_id','stars','user_id')
train_set = construct_set_for_ALS(rating, 9)

In [12]:
rating.select('stars').first()

Row(stars=2)

In [13]:
%%time
train_set.first()

CPU times: user 11.1 ms, sys: 6.3 ms, total: 17.4 ms
Wall time: 1min


Row(user_id='-55DgUo52I3zW9RxkZ-EAQ', business_id='wHq1efQVz17338k_aUOX3w', stars=5, user_count=16)

In [16]:
train_set.cache()

DataFrame[user_id: string, business_id: string, stars: bigint, user_count: bigint]

In [23]:
%%time
train_set = train_set.withColumn('user_num', hash_udf(train_set.user_id) )
train_set = train_set.withColumn('biz_num', hash_udf(train_set.business_id) )
print(train_set.columns)

['user_id', 'business_id', 'stars', 'user_count', 'user_num', 'biz_num']
CPU times: user 2.79 ms, sys: 1.6 ms, total: 4.39 ms
Wall time: 18 ms


In [25]:
train_set.randomSplit?

In [27]:
splits = train_set.randomSplit([0.8, 0.2], seed = 91)

In [28]:
train = splits[0]
test = splits[1]

In [29]:
%%time
als = ALS(rank=10, maxIter=5, seed=0, userCol='user_num', itemCol='biz_num', ratingCol='stars')
model = als.fit(train)

CPU times: user 75.7 ms, sys: 31.3 ms, total: 107 ms
Wall time: 16 s


In [30]:
%%time
model.coldStartStrategy
#### add with value drop

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 9.06 µs


Param(parent='ALS_4afcb9dcb1d85fe191f4', name='coldStartStrategy', doc='strategy for dealing with unknown or new users/items at prediction time. This may be useful in cross-validation or production scenarios, for handling user/item ids the model has not seen in the training data. Supported values: nan,drop.')

In [32]:
%%time
model.save('model1')

CPU times: user 4.08 ms, sys: 2.37 ms, total: 6.45 ms
Wall time: 2.07 s


In [35]:
biz_f = model.itemFactors
user_f = model.userFactors

In [36]:
biz_factors_df =biz_f.toPandas()
user_factors_df = user_f.toPandas()

In [37]:
biz_factors_df.to_csv('biz_factors.csv')
user_factors_df.to_csv('user_factors.csv')

In [38]:
from pyspark.ml.evaluation import RegressionEvaluator

In [39]:
evaluator = RegressionEvaluator(labelCol="stars",predictionCol="prediction")

In [41]:
train_res = model.transform(train)
test_res = model.transform(test)

In [43]:
test_res1 = test_res.na.drop()
test_res2 = test_res.fillna(3.6)

In [44]:
%%time
evaluator.evaluate(train_res), evaluator.evaluate(test_res1), evaluator.evaluate(test_res2)

CPU times: user 183 ms, sys: 74 ms, total: 257 ms
Wall time: 1min 15s


(0.6762947506094206, 1.2856843189657963, 1.2861389882836296)

In [45]:
evaluator?